In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: auto; }</style>"))
# TODO need to ctrl-shift-I after that?? wtf?

In [ ]:
# todo hmm, not sure if I really need it?
# %load_ext autoreload
# %autoreload 2

In [ ]:
import os
from pathlib import Path
pp = str(Path(os.getcwd()).absolute().parent / 'src')
import sys
sys.path.insert(0, pp)

In [ ]:
import my.emfit as emfit
emfit_df = emfit.dataframe()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
from bokeh.models import LinearAxis, Range1d
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource as CDS
import pandas as pd

In [ ]:
# see https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#basic-tooltips
# TODO date is displayed as unix time

from dashboard.sleep import add_daysoff, date_figure
from dashboard.core.bokeh import rolling

df = emfit_df
# TODO if there are Nans, it fails to plot anything?? handle in a generic way, add warnings
df = df[df['error'].isnull()]
df = df.set_index('date')

# todo make index??
# mean = cs.rolling(f'{d}D').mean()

# TODO ugh. figure out how pandas determines the index type..

df.index = pd.to_datetime(df.index, utc=True)
dates = df.index

# TODO reuse functions for adding moving averages? Perhaps they should operate on dataframes to be framework independent?
p = date_figure()
add_daysoff(p, dates=dates)

# TODO https://github.com/bokeh/bokeh/blob/master/examples/app/sliders.py

# todo naming: plot vs figure vs graph?
[g, g7, g30] = rolling(plot=p, x='date', y='avg_hr', df=df, color='blue', legend_label='HR')
g7 .glyph.line_width = 2
g30.glyph.line_color = 'lightblue'
g30.glyph.line_width = 2


p.extra_y_ranges = {'resp': Range1d(start=10, end=25)}
# Addi the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name='resp'), 'right')
rolling(plot=p, x='date', y='respiratory_rate_avg', df=df, color='orange', legend_label='Respiration', avgs=['7D'], y_range_name='resp')

p.legend.click_policy = 'hide'

show(p)

In [ ]:
# todo onclick example https://stackoverflow.com/questions/53825387/python-bokeh-update-scatter-plot-colors-on-callback
# TODO determine delta hr and plot 'dependence' on workdays?
# TODO or plot 'derivative'?
# TODO more frequent ticks?

# hrv = df[['hrv_morning', 'hrv_evening']]

# TODO filter non-nan values??
dfm7d  = df.rolling('7D' ).mean()
dfm30d = df.rolling('30D').mean()

# TODO some hrv evening points are right at zero? careful, maybe worth filtering..
p2 = date_figure(x_range=p.x_range)
[g, g7, g30] = rolling(plot=p2, x='date', y='hrv_morning', df=df, color='green' )
g7 .glyph.line_dash = 'dashed'
g30.glyph.line_width = 2
[g, g7, g30] = rolling(plot=p2, x='date', y='hrv_evening', df=df, color='purple')
g7.glyph.line_dash = 'dashed'
g30.glyph.line_width = 2
add_daysoff(p2, dates=dates)

p2.legend.click_policy = 'hide'
show(p2)
# todo disable individual points by default?

def mins(tt):
    # TODO shit. would just use time() but time doesn't allow wrapping around 24 hours
    # and timedelta is not serializable by plotly...
    mm = tt.hour * 60 + tt.minute
    if tt.hour < 16:
        mm += 24 * 60
    # TODO also careful about utc..
    return mm


ints  = df[['sleep_start', 'sleep_end']].applymap(lambda dt: mins(dt.time()))
# todo maybe instead plot angled lines? then won't need messing with minutes at all? Although still useful to keep 
p3 = date_figure(x_range=p.x_range)
from datetime import time
mint = mins(time(22, 0))
maxt = mins(time(11, 0))
add_daysoff(p3, bottom=mint, top=maxt, dates=dates)
p3.vbar(source=CDS(ints), x='date', width=timedelta(1), bottom='sleep_start', top='sleep_end', color='black', alpha=0.1)


from bokeh.models import FuncTickFormatter, FixedTicker
p3.yaxis.ticker = FixedTicker(ticks=list(range(mint, maxt, 30)))
p3.yaxis.formatter = FuncTickFormatter(code="""
    var hh = Math.floor(tick / 60 % 24).toString()
    var mm = (tick % 60).toString()
    if (hh.length == 1) hh = "0" + hh;
    if (mm.length == 1) mm = "0" + mm;
    return `${hh}:${mm}`
""")

# todo scatter: maybe only display few latest points? not sure how easy it is to achieve?

p3.legend.click_policy = 'hide'
show(p3)

In [ ]:
import my.bluemaestro as bluemaestro
orig_df = bluemaestro.dataframe()

In [ ]:
import datetime
from bokeh.models import ColumnDataSource as CDS

df = orig_df
df = df.sort_index() # FIXME should be fixed in hpi? but also make it defensive
df = df[-40000:]

# TODO 'index must be monotonic??'

print(df[-5:])
m1d  = df[['temp']].rolling('1D' ).mean()

p = figure(x_axis_type='datetime', plot_width=2000)
p.scatter(x='dt', y='temp', source=CDS(data=df ), color='blue', legend_label='Temperature'         )
p.line   (x='dt', y='temp', source=CDS(data=m1d), color='red' , legend_label='Temperature (1d avg)')
# TODO plot frequency of data?? could be even automatic (akin to hpi doctor/stat)

show(p)
# TODO highlight different days?

# TODO hpi: I feel like points duplicate each other without being filtered...
# and probably across different export files?? a bit crap

day_count = df.groupby(df.index.date).count()
print(day_count[:10])

p = figure(x_axis_type='datetime', plot_width=2000)
p.vbar(x='index', top='temp', source=CDS(day_count), width=datetime.timedelta(days=1) * 0.95)
show(p)


In [ ]:
# TODO for temp, maybe heatplot would make more sense? E.g. x axis days y axis time of day. although discontinuity at 0:00 is gonna be annoying.
# could group by weeks?

In [ ]:
# TODO hmm, maybe update regression lines dynamically? Wonder if there is as callback in bokeh
# https://stackoverflow.com/q/53287715/706389 here some sample code
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
# regression tap? some callbacky thing
# http://holoviews.org/reference/streams/bokeh/regression_tap.html

# TODO dynamic regression? https://gist.github.com/darothen/b1538c6571b67ed610669314551d3356

xxx = emfit_df[['avg_hr', 'coverage']].dropna()

# normal.opts(size=10) * hv.Slope.from_scatter(normal)
sc = hv.Scatter((xxx['avg_hr'], xxx['coverage']))
# TODO shit. need to filter nans??
hv.Slope.from_scatter(sc).opts(color='red') * sc.opts(size=10)

In [ ]:
import dashboard.core as C

import pandas as pd
import hvplot.pandas  # noqa
from holoviews import opts


# todo add holiday/non-holiday? or days from holiday? could be interesting
cols = ['coverage', 'avg_hr', 'hrv_change', 'bed_time', 'recovery', 'respiratory_rate_avg']
df = emfit_df[cols]

# todo uhoh. I guess the other fields won't be available for hover?
hover = HoverTool(
    tooltips=[(x, f'@{x}') for x in cols],
    #formatters={
    #    '@date'        : 'datetime', # use 'datetime' formatter for '@date' field
    #},
    # display a tooltip whenever the cursor is vertically in line with a glyph
    # not sure about that here
    # not sure if it even works??
    # mode='vline'
)


# todo reuse/determine width
# hmm, didn't manage to quickly google how to do this
tools = [hover, 'box_select', 'lasso_select', 'tap']
sm = C.scatter_matrix(df, width=2000, height=2000).opts(
    opts.Scatter  (tools=tools), 
    opts.Histogram(tools=tools),
#    # opts.Layout(width=2000, height=2000), # -- doesn't seem to do anything?
)
sm


# TODO add tools to do vertical projections?


# ugh. https://github.com/holoviz/hvplot/blob/ab43c4f68aa7e485326dea567a348b96d24ebf60/hvplot/plotting/scatter_matrix.py#L45 not sure if the needed parameters are passed?..

# gridmatrix code? https://github.com/holoviz/holoviews/blob/5deb4a7b7f04f989fcc39bb06e657e8b6dfb4ea2/holoviews/operation/element.py#L997

# TODO date slider? to hightligh changes over dates E.g. left-right
# TODO show dates on tooltips

In [ ]:
from dashboard.core import _scatter_matrix_demo
sm = _scatter_matrix_demo(width=2000)
sm

In [ ]:
# some examples of server with interactive stuff? 
# https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/main.py

# these for sleep intervals? https://docs.bokeh.org/en/latest/docs/gallery/stacked_area.html
# also I had some old jawbone plots I think...
# also existing emfit plots

# food plots (breakdown) https://docs.bokeh.org/en/latest/docs/gallery/bar_stacked.html

# TODO errors should be aggregated and displayed as a separate column? with ability to toggle


# TODO ok, plotly has some callbacks for lasso/rectangle selection https://plotly.com/python/v3/selection-events/
# with these, I could dynamically update the regression lines, I guess


# TODO some nice dynamic interactions https://docs.bokeh.org/en/0.10.0/docs/user_guide/interaction.html

In [ ]:
import my.body.blood as B
bdf = B.dataframe()

In [ ]:
p = date_figure()
# TODO fuck. seems that bokeh fails to plot if some data is interleaved with Nans
# right, see https://github.com/bokeh/bokeh/issues/737
# I guess, need a helper after all to handle it better

# TODO set x range as the first thing to max
x_range = None

cols = set(bdf.columns)
cols = cols.difference({'dt', 'extra', 'source'})

# todo autodiscover fields??
for f in sorted(cols):
    # todo add source to annotation?
    # todo add normal ranges
    p = date_figure(x_range=x_range)
    
    fdf = bdf[bdf[f].notna()]
    
    p.scatter(x='dt', y=f, source=CDS(data=fdf))
    p.line   (x='dt', y=f, source=CDS(data=fdf))
    p.title.text = f
    show(p)
    if x_range is None:
        x_range = p.x_range

In [ ]:
from dashboard.data import locations_dataframe
locs = locations_dataframe()

In [ ]:
import pandas as pd
import numpy as np
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
from bokeh.io import output_notebook
output_notebook()
pd.set_option('display.max_columns', 100)
gv.extension('bokeh')

In [ ]:
from datetime import date
from_ = date(2019, 2, 1)
to    = date(2019, 4, 1)
df = locs[(from_ <= locs.index.date) & (locs.index.date <= to)] # meh, but not too bad

df = df[['lat', 'lon']]
df['date'] = df.index.date # meh
df[:5]
# todo wonder, what's its performance? how many points it can handle?
# ok, 2 month of data took about a minute to render

In [ ]:
# https://malouche.github.io/notebooks/geoviews2.html
# https://data-dive.com/interactive-maps-made-easy-geoviews
plot = (gf.ocean * gf.land * gf.coastline * gf.borders).options(
    'Feature', projection=crs.Mercator(), global_extent=True, frame_height=1000,frame_width=1500,
)
# Convert locations to points and overlay on previous map
points = gv.Points(df, kdims=['lon', 'lat'], vdims=['date']).groupby(['date'])
plot_points = plot * points.opts(alpha=0.3, color='red', size=4)
plot_points.opts(width=500, height=400)
# todo hmm, how to set initial zoom? ideally, auto adjust it interactively depending on the day
# todo how to add proper keybindings for move/zoom? with ctrl etc